In [67]:

import os
import sys
import datetime
import pandas as pd

#local imports
sys.path.append("..")
import simulator.utils.translate_to_fb as fb_utils
import simulator.constants as con

In [68]:
data = os.listdir(os.path.join(con.DATA_FOLDER))
sample_data = data[0]

df_data = pd.read_csv(os.path.join(con.DATA_FOLDER, sample_data),
                      parse_dates=["timestamp", "date"], dtype={"identifier": str})
df_data.drop(columns="province_short", inplace=True)
df_data.head()

,identifier,timestamp,date,device_lat,device_lon
0,a869f588-d7e3-40aa-951c-7352154a7c4c,2020-02-01 15:27:13,2020-02-01,4.964664,-73.916862
1,75e311c4-9c2b-43c2-85f2-c7c25f9903ad,2020-02-01 23:37:04,2020-02-01,4.782922,-74.761330
2,4619d188-6cd4-487b-baf8-40ac83789a20,2020-02-01 14:30:58,2020-02-01,4.505649,-73.349306
3,be8e1fb8-d3ca-4067-83ae-5e3de312cd1d,2020-02-01 17:58:23,2020-02-01,4.721836,-74.215469
4,2860106b-2be2-4c8a-8c27-901308cd04ae,2020-02-01 02:39:19,2020-02-01,4.607332,-74.213275


In [69]:
# Calculate quadkey
df_data["quadkey"] = fb_utils.extract_quad_keys(df_data[["device_lat", "device_lon"]].to_numpy())

In [70]:
def to_fb_date(t):
    fb_date = t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
    if t.hour >= 16 and t.hour < 24:
        fb_date = t.replace(second=0, microsecond=0, minute=0, hour=16)
    elif t.hour >= 0 and t.hour < 8:
        fb_date = t.replace(second=0, microsecond=0, minute=0, hour=0)
    elif t.hour >= 8 and t.hour < 16:
        fb_date = t.replace(second=0, microsecond=0, minute=0, hour=8)
    return fb_date


In [71]:
# Set time intervals
df_data["fb_datetime"] = df_data["timestamp"].apply(to_fb_date)


# If the same person appeared at multiple locations in a time interval we only count their most frequent location.
df_data.sort_values("fb_datetime", inplace=True)
before = df_data.shape[0]
df_data.drop_duplicates(subset=["identifier", "fb_datetime"], 
                        keep="last", inplace=True)
after = df_data.shape[0]
print(f"Dropped {before - after} duplicates.")


Dropped 19767 duplicates.


In [72]:
df_data.drop(columns=["identifier", "timestamp", "date", 
                      "device_lat", "device_lon"], inplace=True)
df_data["population"] = 1
df_data.head()


,quadkey,fb_datetime,population
71726,33212133211212,2020-02-01,1
71769,33212133211212,2020-02-01,1
72589,33212133211212,2020-02-01,1
73238,33212133211212,2020-02-01,1
72591,33212133211212,2020-02-01,1


In [73]:
df_data = df_data.groupby(["quadkey", "fb_datetime"])["population"].sum().reset_index()

In [74]:
df_data.head()

,quadkey,fb_datetime,population
0,33212133003321,2020-02-07 08:00:00,1
1,33212133003321,2020-02-10 16:00:00,1
2,33212133003321,2020-03-01 08:00:00,1
3,33212133003321,2020-03-02 08:00:00,1
4,33212133003321,2020-03-08 08:00:00,1
